## <center> 🌍 План ноутбука

<img src="images/goose_intro.png" align="right" width="400">

0. [🚗 Описание задачи и знакомство с данными.](#part0)
1. [⚙️ Генерация и фильтрация признаков.](#part1)
2. [🔮 Прогнозируем время и вид поломки машин - настраиваем ML модель.](#part2)
3. [📊 Визуализация прогнозов, ошибок модели и важности признаков.](#part3)
4. [🔄 Оптимизация. Тюнинг гиперпараметров с помощью `Optuna`.](#part4)
5. [🤹‍♀️ Блендинг.](#part5)
6. [🏆 Засылаем финальное решение на лидерборд на Kaggle.](#part6)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## <center id="part0"> 🚗 Описание задачи

**Предыстория:**

- Мы работаем с каршеринговой компанией, управляющей крупным автопарком машин.
- Цель: предотвратить длительные периоды простоя машин из-за поломок через своевременное обслуживание и ремонт. 

**Идея для решения проблемы:**

- Создание приоритезированного списка обхода машин. 
- Этот список поможет технической бригаде сосредоточиться на автомобилях, которые наиболее вероятно выйдут из строя в ближайшее время.

**Как будем решать?:**

- Компания собирает данные о поездках и состоянии машин до поломок. 
- Компания наняла Data Scientist'а, чтобы он смог использовать эти данные для анализа и прогнозирования характера поломок.

**Важный момент**

- Роль этого специалиста (Data Scientist) будешь играть ты! 😎

## <center> 🔎 Знакомство с данными

<center> <img src="images/data_tables_schem.JPG" width=400>

### Информация про машины с таргетом (основной датасет)

In [2]:
car_train = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/car_train.csv')
car_train.sample(5)

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
549,t41251369W,Hyundai Solaris,economy,petrol,3.12,2011,2664,2016,64.78,another_bug
1242,q13192264P,Renault Kaptur,standart,petrol,4.70,2012,24874,2021,51.22,engine_check
986,N29840074e,Smart ForTwo,economy,petrol,4.52,2015,79976,2020,24.73,engine_overheat
590,v77404713z,Nissan Qashqai,standart,petrol,3.28,2015,88356,2019,42.45,electro_bug
267,F72202302t,Smart ForTwo,economy,petrol,3.88,2017,106430,2019,51.85,engine_check


- `car_id` — идентификатор машины
- `model` / `car_type` / `fuel_type` — марка, класс и тип топлива машины
- `car_rating` / `riders` — общий рейтинг и общее число поездок к концу 2021-го года
- `year_to_start` / `year_to_work` — год выпуска машины и год начала работы в автопарке
- `target_reg` — количество дней до поломки
- `target_class` — класс поломки (всего 9 видов).

### Информация про поездки

In [3]:
rides_info = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/rides_info.csv')
rides_info.sample(5)

,user_id,car_id,ride_id,ride_date,rating,ride_duration,ride_cost,speed_avg,speed_max,stop_times,distance,refueling,user_ride_quality,deviation_normal
714145,c13653279A,y-2901647k,E1H,2020-01-27,3.34,53,523,43,76.0,1,2096.180243,0,0.593899,1.232
344498,Y22780579z,Y22302267X,m1g,2020-03-21,0.10,45,401,33,67.0,0,1464.970892,0,6.353186,0.001
74995,r22731075S,F-6369826L,v1I,2020-01-01,2.70,17,129,50,88.0,2,439.228596,0,-0.290889,-1.770
277338,v33411671Q,T-3474422m,a1a,2020-03-23,4.17,51,454,49,61.0,1,1738.436174,0,-13.606936,-17.071
330777,F10550038H,X15138491s,D1K,2020-01-02,1.23,69,890,41,144.0,1,2782.099733,0,1.615149,-0.000


- `user_id` / `car_id` / `ride_id` - идентификаторы водителя, машины, поездки соответственно
- `ride_date` / `rating` - дата поездки и рейтинг, поставленный водителем
- `ride_duration` / `distance` / `ride_cost` -  длительность (время),  пройденное расстояние, стоимость поездки
- `speed_avg` / `speed_max` - средняя и максимальная скорости поездки соответственно
- `stop_times` / `refueling` - количество остановок (паузы) и флаг - была ли дозаправка.
- `user_ride_quality` - оценка манеры вождения водителя машины, определенная скоринговой ML системой сервиса.
- `deviation_normal` - общий показатель датчиков о состоянии машины, относительно эталонных показателей (нормы).

### Информация про водителей

In [4]:
driver_info = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/driver_info.csv')
driver_info.sample(5)

,age,user_rating,user_rides,user_time_accident,user_id,sex,first_ride_date
6355,19,6.8,2015,4.0,Q15095157g,1,2018-3-6
4499,30,8.1,1459,9.0,O17985654p,1,2018-8-1
13770,49,8.0,1967,5.0,m13273997W,0,2018-3-17
13838,45,7.4,1877,7.0,Q51539900B,0,2021-9-17
13888,36,8.6,420,1.0,O16710533I,0,2019-7-25


- `user_id` / `age` / `sex` — идентификатор, возраст и пол водителя, соответственно
- `user_rating` — общий рейтинг пользователя за все поездки к концу 2021-го года
- `user_rides` — общее количество поездок к концу 2021-го года
- `user_time_accident` — число инцидентов (это могли быть аварии/штрафы/эвакуация машины)  
- `first_ride_date` — дата первой поездки.

### Информация про ремонт машин

In [5]:
fix_info = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/fix_info.csv')
fix_info.sample(5)

,car_id,worker_id,fix_date,work_type,destroy_degree,work_duration
114253,x-1111076W,KO,2019-9-27 20:34,repair,5.7,18
3432,d53962651M,RM,2017-4-3 4:22,repair,5.0,34
23982,f17933532Q,TB,2020-12-29 6:44,repair,5.3,16
37169,G-1056464Y,HS,2020-3-9 20:51,repair,5.9,21
11343,x-8009442a,UV,2018-9-7 8:1,reparking,1.0,26


- `worker_id` / `car_id` — идентификатор работника и машины;
- `work_type` / `work_duration` — тип и длительность (в часах) проводимой работы;
- `destroy_degree` — степень износа/повреждённости машины в случае поломки;
- `fix_date` — время начала ремонта (снятия машины с линии).

## <center id="part1">⚙️ 1. Генерация и фильтрация признаков.

<center> <img src="https://ucarecdn.com/bf4c772d-b67a-42ae-a48b-cfd83910b0a2/" width=700>

<div class="alert alert-info">

**Цель блока** - сгенерировать признаки из дополнительных датасетов и добавить их к основному датасету, произвести фильтрацию признаков.

    
<center> <h3> Задание 1. Генерация признаков из дополнительных датасетов [6 баллов]</h3>

<div class="alert alert-info">

1. Возьмите датасет `rides_info` с информацией о поездках и проведите группировку по каждой машине отдельно.

2. Для каждой машины (то есть для каждого `car_id`) подсчитайте несколько признаков:
   - Какой минимальный рейтинг за все поездки.
   - Какой средний рейтинг за все поездки
   - Сколько всего километров проехала.
   - Какая была максимальная скорость.
   - Сколько всего поездок сделала каждая машина.
   - Добавьте еще минимум 3 признаки на свой выбор.

3. Сделайте соединение таблиц, вспомнив про методы соединения и выбрав подходящий для нашего случая.

4. Подключите информацию про водителей (`driver_info`) и про ремонт машин (`fix_info`).
    - Добавьте минимум по 3 признака на свой выбор с каждого из датасетов  


<h4> Критерии оценивания </h4>
    
- Были добавлены 5 обязательных и минимум 3 на свой выбор признака из датасета `rides_info` [2 балла]
- Были добавлены минимум 3 признака на свой выбор из датасета `driver_info` [2 балла]
- Были добавлены минимум 3 признака на свой выбор из датасета `fix_info` [2 балла]

</div>

<div class="alert alert-success">
    
<h4> Несколько советов по Feature Engineering</h4>
    
- 🏁 Начинайте с сырых данных. 
- 🧩 Используйте все доступные данные. Покрывайте признаками всю имеющуюся информацию в данных.
- 🧠 Формулируйте предположения: от чего зависит таргет? 
- 🤔 Смотрите визуально на классы/ошибки и делайте предположения. Какие полезны?
- ⚠️ Помните, что слишком много признаков может быть вредно. Потом придется отфильтровывать.
    
<h4> Полезные ссылки </h4>
    
- [Ноутбук](https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/3.2%20-%20Feature_Engineering.ipynb) про генерацию признаков
- [Ноутбук](https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/3.3%20-%20Feature%20Selection.ipynb) про фильтрацию признаков
- [Большой и подробный гайд](http://www.feat.engineering/) (на английском)

In [6]:
# Пример расчета одного признака
rides_info.groupby('car_id', as_index=False).agg(
    mean_rating = ('rating', 'mean'),
    # ... еще признаки
)

df = pd.merge(...) # Соедините полученный датасет с фичами с основным датасетом

# YOUR CODE HERE

TypeError: merge() missing 1 required positional argument: 'right'

Идеи для новых признаков. Подумайте, какие из них самые полезные?
    
* `feature_min_max_diff`: разница между максимальным и минимальным значениями `deviation_normal` для каждой машины
* `feature_corner`: угол наклона по признаку `user_ride_quality` для каждой машины
* `feature_mean`: среднее значение `deviation_normal` для каждой машины
* `feature_shift`: точка перегиба/сдвига для `deviation_normal`
* `feature_start`: значение точки старта для `deviation_normal`
* `feature_nans`: сумма пропусков для столбца `...` для каждой машины 
* `feature_quant`: `X %` квантиль для столбца `...` для каждой машины

In [ ]:
# YOUR CODE HERE

### <center> 🧹 Нагенерировали? А теперь будем убирать!

<center> <img src="https://ucarecdn.com/d1b4bc78-fd04-44fb-bdbf-0a63355b7384/" width=700>

<div class="alert alert-info">

<h3><center> Зачем вообще заниматься отбором признаков?</center></h3>
    
**Основные причины**:
    
1. Если фичей очень много, то данные могут перестать помещаться в память, может существенно увеличиться время обучения модели, тем более если мы захотим протестировать несколько разных алгоритмов или ансамбль. Особенно, в условиях ограничения платформ на длительность одной сессии (в Kaggle 12 часов) и лимиты по потребляемой памяти.
2. Главная причина: с увеличением количества признаков часто падает точность предсказания модели. Особенно, если в данных большое количество мусорных фичей (почти не коррелирующих с таргетом). Некоторые алгоритмы при сильном увеличении числа признаков, вообще, перестают адекватно работать. И здравствуй, старый добрый оверфит!
3. Даже если точность не снижается, есть риск, что ваша модель опирается на шумные фичи, что снизит стабильность прогноза на приватной выборке.

<h3> <center> Несколько советов по Feature Selection </center> </h3>
    
<b>Что можно удалить сразу?</b>
    
- Константы
- Уникальные значения (в том числе в тесте, как правило это ID-шники по типу `car_id`)
    
<b>Какие методы использовать дальше?</b>
    
- `Линейная корреляция`
- `Phik` - тоже корреляция, но на стероидах ([туториал на Medium](https://towardsdatascience.com/phik-k-get-familiar-with-the-latest-correlation-coefficient-9ba0032b37e7))
- `Permutation importance` (из `scikit-learn`)
- `SHAP values` (из библиотеки `shap`)
- `CatBoost Feature Selection` (рекурсивные методы)
    

<center> <h3> Задание 2. Применение методов фильтрации признаков [4 балла]</h3>

1. Выберите и примените любые 3 (можно и больше) метода фильтрации, которые указаны выше или в ноутбуке по фильтрации из полезных ссылок.
2. Проинтерпертируйте результаты и сделайте выводы.


<h4> Критерии оценивания </h4>
    
- Применены методы фильтрации [1 балл за каждый метод, максимум 3 балла].
- Сделаны выводы на основе примененных методов [1 балл]
</div>

In [ ]:
# YOUR CODE HERE

## <center id="part2"> 🔮 2. Прогнозируем время и вид поломки машин - настраиваем ML модель.

<div class="alert alert-info">

**Цель блока** - составить тренировочную и валидационную выборки, произвести обучение модели `CatBoostClassifier` на тренирочной части и оценить качество на валидационной.

<center> <h3> Задание 3. Обучение первой модели [5 баллов]</h3>

1. Классифицируйте признаки на типы (категориальные, числовые, таргеты)

2. Выделите в `X` только признаки, а в `y` таргет (для задачи классификации).

3. Сделайте разделение данных на *обучающую* и *валидационную* выборки (не забывайте про воспроизводимость ваших результатов).

4. Создайте и обучите `CatBoostClassifier` модель (настраивать гиперпараметры сейчас не обязательно).

5. Проведите оценку вашей модели, используя метрику `accuracy`.


<h4> Критерии оценивания </h4>
    
- По 1 баллу за каждый корректно выполненный пункт.

</div>

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
features2drop = [...] # то, что надо выбросить
targets = [...] # таргеты
cat_features = [...] # категориальные признаки

num_features = [...] # числовые признаки

print('Категориальные признаки:', len(cat_features), cat_features)
print('Числовые признаки:', len(num_features), num_features)
print('Целевые переменные', targets)

In [ ]:
# YOUR CODE HERE

## <center id="part3"> 📊 3. Визуализация прогнозов, ошибок модели и важности признаков.

<div class="alert alert-info">

**Цель блока** - визуализировать результаты прогнозирования, ошибки модели и важность признаков для лучшего понимания и анализа модели.

<center> <h3> Задание 4. Визуализация [5 баллов]</h3>

   
1. Визуализируйте важность признаков встроенным методом в `CatBoost` (помните, что не стоит сильно доверять этому рейтингу важности, т.к. для сильно скоррелированных признаков важность делится пополам и оба признака могут улететь вниз по важности).

2. Постройте `waterfall_plot` из библиотеки `SHAP` (подробнее во втором ноутбуке из полезных ссылок ниже).
    
3. Постройте `classification_report` из библиотеки `scikit-learn`.

4. Постройте и визуализируйте матрицу смежности (`confusion_matrix`), посмотрите в каких классах модель больше всего ошибается.
    
5. Для каждого графика/примененного метода проинтерпретируйте результаты и сделайте выводы.


<h4> Критерии оценивания </h4>
    
- По 1 баллу за каждый корректно выполненный пункт.
    
</div>

<div class="alert alert-success">

Визуализация может помочь даже после того, как мы **уже обучили** какую-нибудь модель. Например:

- Понять, что мешает модели или чего не хватает, чтобы не допускать ошибки
- Сделать выводы, как можно улучшить точность в последующих экспериментах.
- Визуализировать ошибки модели
- Отсеять лишние признаки
- Найти идеи для новых признаков.
- Все зависит от типа ваших данных

<h3> Полезные ссылки </h3>
    
- [Ноутбук](https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/2.3%20-%20Visualisation.ipynb) про визуализацию и `seaborn`.
- [Ноутбук](https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/3.4%20-%20Visualisation.ipynb) про продвинутую визуализацию и анализ ошибок модели
</div>    

In [ ]:
# YOUR CODE HERE

## <center id="part4"> 🔄 4. Оптимизация. Тюнинг гиперпараметров с помощью `Optuna`

<center>
    <img src="images/goose_tuning.png" width="500">
</center>

<div class="alert alert-info">

<h3>Ключевые особенности <code>Optuna</code>:</h3>

- 🎯 Легковесность и универсальность - можно подбирать оптимальные параметры под любые функции и метрики
- 🎁 SOTA алгоритмы, адаптированные для поиска гиперпараметров
- ⏱ Параллелизация и различные методы прунинга
- 📈 Встроенная визуализация
- 🤝 Интеграция со множеством популярных библиотек (бустинги, sklearn, PyTorch, W&B и другие)

Чтобы понять, как ее использовать, давайте разберем ее по частям.


> [Ноутбук](https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/5.3%20-%20Optuna.ipynb) по `Optuna`

### В `Optuna` присутствуют 2 базовые сущности:

<div class="alert alert-info">

<h4> <code>Study</code>: оптимизация, базирующаяся на <code>Objective</code> функции.</h4>

В `Objective` функцию нужно написать код подсчета метрики, которую возвращаем. `Objective` вызывается Optun'ой много раз для подбора лучших параметров.
```python
def objective(trial, ...):
    # calculate score...
    return score
```

<h4> <code>Trial</code> - одно выполнение <code>Objective</code> функции.</h4>

В `trial` обьекте мы передаем параметры для "перебора", используя для каждого типа свой метод. К примеру

```python
# метод `suggest_float` показывает, что перебираем `float` значения, от 0 и до 1.5 границы.
param = trial.suggest_float('param', 0, 1.5) 

# Категориальное значение
loss_function = trial.suggest_categorical('loss', ['Logloss', 'CrossEntropy'])

# Целочисленное значение
depth = trial.suggest_int('depth', 5, 8)

# Равномерное распределение
learning_rate = trial.suggest_uniform('learning_rate', 0.0, 1.0)
```

### Study parameters

<div class="alert alert-info">

Инициализируем обьект `study`, который начнет перебор и сохранит в себе историю результатов.
Если мы стараемся увеличить метрику, а не уменьшить ошибку, то используем `create_study(direction='maximize')` 
```python
study = optuna.create_study()
study.optimize(objective, n_trials=10)
```

<div class="alert alert-info">
    
В [`Optuna`](https://optuna.readthedocs.io/en/stable/index.html) реализовано несколько методов (`sampler`) подбора параметров (в том числе классические):
* `GridSampler`
* `RandomSampler`
* `Tree-Structed Parzen Estimator` (`TPESampler` - самый популярный - дефолтный)
* `BruteForceSampler`
* И ещё [4 других](https://optuna.readthedocs.io/en/stable/reference/samplers/index.html#module-optuna.samplers), также можно написать собственный сэмплер.

<div class="alert alert-info">

<h3> 🌀 Советы по перебору параметров (очень логичные). </h3>
    
- Иметь понимание важности параметров
- Число `iterations` лучше взять с запасом и зафиксировать, при этом ограничив через `early_stopping_rounds`
- Подсмотреть/чувствовать диапазоны и шаг значений
- Исключить то, что перебирать не нужно. (`random_seed` , `eval_metric`, `thread_count` и прочее)
- Используйте информацию с прошлых попыток

<div class="alert alert-info">

**Цель блока** - улучшить качество предсказания, произведя подбор гиперпараметров для модели с помощью `Optuna`

<center> <h3> Задание 5. Подбор гиперпараметров [3 балла]</h3>

1. Напишите `objective` функцию и запустите `Optuna`.
2. Подберите гиперпараметры для `CatBoostClassifier` (минимум 3 гиперпараметра).
3. Обучите модель с новыми гиперпараметрами, сравните качество и сделайте выводы.

    
<h4> Критерии оценивания </h4>
    
- По 1 баллу за каждый корректно выполненный пункт.
    
</div>

In [ ]:
# YOUR CODE HERE

## <center id="part5"> 🤹‍♀️ 5. Блендинг

> Одна голова хорошо, а `n` голов всё же лучше

Если на этапе построения первой модели (когда мы выбрали `CatBoost`) вы задались вопросом "А почему он, а не ~~я~~ `LightGBM` или `XGBoost` (или даже более простые модели)?", то сейчас вы можете построить всё, что пожелаете. А дальше, мы построим ансамбль этих моделей - **блендинг**. 

Самый простой способ сделать **блендинг** — это усреднить предсказания или взвесить ответы.

    
<center>
    <img src="images/goose_blending.png" width="500">
</center>


<div class="alert alert-success">
    
<h3> Зачем нужен блендинг? </h3>
    
* Основная идея блендинга - взять от каждого алгоритма лучшее и совместить несколько разных ML моделей в одну. 
* За счет такого объединения увеличивается обобщающая способность финальной модели и качество улучшается.
* Помимо этого, ваша модель становится более стабильной, что позволяет не слететь на приватном лидерборде.
* Особенно хорошо накидывает блендинг, если смешиваемые **модели имеют разную природу**: например, нейронные сети, KNN и решающие деревья, в этом случае они выучивают разные зависимости и хорошо дополняют друг друга.
    
<h3> Полезные ссылки </h3>
    
- [Ноутбук](https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/6.1%20-%20Blending.ipynb) по блендингу
- [Ноутбук](https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/4.1%20-%20CatBoost.ipynb) по `CatBoost`
- [Ноутбук](https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/4.2%20-%20LightGBM.ipynb) по `LightGBM`
- [Ноутбук](https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/4.3%20-%20XGBoost.ipynb) по `XGBoost`

</div>

<div class="alert alert-info">

**Цель блока** - улучшить качество предсказания, объединив несколько моделей вместе методом блендинга.

<center ><h3> Задание 7. Блендинг [10 баллов]</h3>

1. Построить и обучить модели
    - `CatBoostClassifier`
    - `LightGBMClassifier (goss)`
    - `XGBoostClassifier (dart)`
    - `RandomForestClassifier`
2. Сделать предсказания каждой моделью, оценить качество.
3. Реализовать блендинг двумя способами, оценить качество и сравнить с предыдущим пунктом
    - `Hard Voting` - метод, при котором мы делаем голосование всех моделей и выбираем самый популярный класс.
    - `Soft Voting` - метод, при котором мы складываем вероятности предсказания всех моделей по каждому классу и потом выбираем самый класс с максимальной суммой.
    
<h4> Критерии оценивания </h4>
    
- По 1 баллу за каждую обученную модель [4 балла]
- По 0.5 балла, если для модели проводился подбор гиперпараметров [2 балла]
- Сделаны предсказания каждой модели и оценено качество [1 балл]
- Реализован `Hard Voting` [1 балл]
- Реализован `Soft Voting` [1 балл]
- Сделаны выводы [1 балл]    
    
</div>

In [ ]:
# YOUR CODE HERE

## <center id="part6"> 🏆 6. Засылаем финальное решение на лидерборд на Kaggle

<div class="alert alert-info">

**Цель блока** - сделать предсказание с помощью блендинга для тестовой части датасета, отправить результат в [соревнование на Kaggle](https://www.kaggle.com/competitions/competative-data-science-course-by-data-feeling/overview).

<center> <h3> Задание 8. Предсказание на тестовом датасете и отправка на Kaggle</h3>

1. Сделать предобработку для тестового датасета, присоединить к нему информацию из других датасетов и добавить признаки, которые генерировали для тренировочного датасета.
2. Сделать предсказания каждой моделью (которые хотите включить в ансамбль, но минимум их должно быть 3)
3. Сделать блендинг (с помощью `Hard Voting` или `Soft Voting` на ваш выбор)
4. Сохранить результат предсказания в `csv` файл и отправить решение на Kaggle.
    
<h4> Критерии оценивания </h4>
    
- 0 баллов за задание, если итоговый скор на лидерборде меньше, чем `0.9`
- По 1 баллу за каждый выполненный пункт

</div>

In [ ]:
car_test = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/car_test.csv')
car_test.head(3)

In [ ]:
# YOUR CODE HERE

Если пишите код на `Kaggle`, то для отправки решения:

- Нажимаем `Save Version`
- Проверяем, что в `Advanced Settings` выбрано - всегда сохранять результаты ноутбука.
- Тип сохранения `Save & Run All (Commit)`
- Жмем окончательно кнопку `Save`

<center>
    <img src="images/goose_winner.png" width="500">
</center>